In [1]:
import numpy as np
import nltk
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout

In [2]:
train = pd.read_csv("train.csv")

In [3]:
data = train[['text','target']].copy()

In [4]:
def removeURL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

data["text"]=data["text"].apply(lambda x: removeURL(x))

In [5]:
def removeEMO(text):
    pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return pattern.sub(r'', text)
data["text"]=data["text"].apply(lambda x: removeEMO(x))

In [6]:
def removePUN(text):
    return text.translate(str.maketrans('', '', string.punctuation))
data["text"]=data["text"].apply(lambda x: removePUN(x))

In [7]:
s = 7000

In [8]:
onehot = [one_hot(words,s) for words in data['text']]

In [9]:
sent_length =40
X = pad_sequences(onehot,padding='pre',maxlen=sent_length)
embedding_vector_features=50

Y = data['target']

In [22]:
model=Sequential()
model.add(Dense(100,activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(50,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=1)

In [24]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=30,batch_size=32)

Epoch 1/30
179/179 [==============================] - 1s 4ms/step - loss: 0.6814 - accuracy: 0.5609 - val_loss: 0.6820 - val_accuracy: 0.5720
Epoch 2/30
179/179 [==============================] - 1s 3ms/step - loss: 0.6773 - accuracy: 0.5617 - val_loss: 0.6768 - val_accuracy: 0.5730
Epoch 3/30
179/179 [==============================] - 0s 3ms/step - loss: 0.6743 - accuracy: 0.5696 - val_loss: 0.6776 - val_accuracy: 0.5572
Epoch 4/30
179/179 [==============================] - 0s 3ms/step - loss: 0.6756 - accuracy: 0.5644 - val_loss: 0.6769 - val_accuracy: 0.5683
Epoch 5/30
179/179 [==============================] - 0s 3ms/step - loss: 0.6758 - accuracy: 0.5656 - val_loss: 0.6754 - val_accuracy: 0.5625
Epoch 6/30
179/179 [==============================] - 0s 3ms/step - loss: 0.6766 - accuracy: 0.5540 - val_loss: 0.6893 - val_accuracy: 0.5730
Epoch 7/30
179/179 [==============================] - 0s 3ms/step - loss: 0.6745 - accuracy: 0.5684 - val_loss: 0.6804 - val_accuracy: 0.5462
Epoch 

In [31]:
Y_pred = model.predict_classes(X_test)

In [32]:
accuracy_score(Y_test,Y_pred)

0.542016806722689